In [2]:
%load_ext Cython

In [85]:
import pandas as pd

In [93]:
num = 500
y = np.random.randint(2, size=(num, 1))
x = np.random.randint(10, size=(num, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

王然老师初步优化代码

In [94]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0]) 
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [95]:
%%timeit
target_mean_v2(data,'y','x')

10 loops, best of 3: 27.3 ms per loop


Cython版

In [ ]:
%%cython -a
import numpy as np
cimport numpy as cnp

cpdef cnp.ndarray target_mean_v3(cnp.ndarray data_x, cnp.ndarray data_y):
    cdef int row_num = data_x.shape[0]
    cdef cnp.ndarray result = np.zeros(row_num)
    cdef cnp.ndarray data_value = np.zeros(10)
    cdef cnp.ndarray data_count = np.zeros(10)
    cdef int i = 0 
    for i in range(row_num):
        data_value[data_x[i]] += data_y[i]
        data_count[data_x[i]] += 1
    for i in range(row_num):
        result[i] = (data_value[data_x[i]] - data_y[i]) / (data_count[data_x[i]] - 1)
    return result

In [96]:
%%timeit
target_mean_v3(data['x'].values,data['y'].values)

1000 loops, best of 3: 1.11 ms per loop


In [97]:
result_3 = target_mean_v3(data['x'].values,data['y'].values)
result_2 = target_mean_v2(data,'y','x')
np.linalg.norm(result_2-result_3)

0.0